In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -U sentence-transformers
! pip install fugashi
! pip install ipadic

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 2.8 MB 31.0 MB/s 
     |████████████████████████████████| 1.2 MB 39.4 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
     |████████████████████████████████| 636 kB 54.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=e77556b016777d8fac36959f610b4f380c0caa6cf3b3db2a1da877051bb7bb13
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 490 kB 5.3 MB/s 
     |████████████████████████████████| 13.4 MB 207 kB/s 
  Cre

### Loading Libraries

In [ ]:
import pandas as pd

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

from sentence_transformers import SentenceTransformer
from transformers import BertJapaneseTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util

import torch

from sklearn.metrics import accuracy_score


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  This is separate from the ipykernel package so we can avoid doing imports until


### Load Data

In [ ]:
filepath = "/content/drive/MyDrive/Macnica/Projects/STS/"
xls =  pd.ExcelFile(filepath + "data/dataset_JA_Clinical_STS.xls")
df = xls.parse(0)

In [ ]:
df.columns = ['s1', 's2', 'score']

In [ ]:
y = df['score']
X = df.drop(['score'], axis = 1)

In [ ]:
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)



In [ ]:
from sentence_transformers import SentenceTransformer, util
model1 = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model2 = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model3 = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model4 = SentenceBertJapanese('sonoisa/sentence-bert-base-ja-mean-tokens')
model5 = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertJapaneseTokenizer'.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=13)

In [ ]:
from cleaning import *
X_train['s1'] = cleaning(X_train['s1'])
X_train['s2'] = cleaning(X_train['s2'])

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

...

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

  0%|          | 0/2935 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
n = '5'
exec(f'model = model{n}')

In [ ]:
#Compute embedding for both lists
#Change models numbers according to the need
embeddings1 = model.encode(df['s1'])
embeddings2 = model.encode(df['s2'])

#Compute cosine-similarities
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
#Creating an empty pred column to add prediction scores
df['cs'+n] = ''
for i in range(len(df['s1'])):
    df['cs' + n][i] = cosine_scores[i][i].item()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
df['cs'+n] = df['cs'+n].apply(lambda x: int(round(x*5, 0)))

print("Accuracy of cs"+ n + ":", accuracy_score(df['score'], df['cs'+n]))

Accuracy of cs5: 0.24666121559007903


## Retraining Models

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses

score = y_train / 5  # Normalize score to range 0 ... 1
train_examples = InputExample(texts=[X_train['s1'], X_train['s2']], label=score)

train_dataset = SentencesDataset(train_examples, model)


In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses


train_samples = []
#score = y_train / 5  # Normalize score to range 0 ... 1
for i in range(X_train.shape[0]):
  score = float(y_train.iloc[i])/5.0
  inp_example = InputExample(texts=[X_train['s1'].iloc[i], X_train['s2'].iloc[i]], label=score)
  train_samples.append(inp_example)



In [ ]:
from torch.utils.data import DataLoader
import logging
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import gzip
import csv

train_batch_size = 16
num_epochs = 4

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator(X_train['s1'], X_train['s2'], y_train)



# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],evaluator=evaluator,epochs=num_epochs,evaluation_steps=1000,warmup_steps=warmup_steps)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/184 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
#Compute embedding for both lists
embeddings1 = model.encode(X_test['s1'])
embeddings2 = model.encode(X_test['s2'])

#Compute cosine-similarities
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
X_test['cs'+n] = ''
for i in range(len(X_test['s1'])):
    X_test['cs'+n][i] = cosine_scores[i][i].item()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
from sklearn.metrics import accuracy_score
df['cs'+n] = df['cs'+n].apply(lambda x: int(round(x*5, 0)))

print("Accuracy of cs" + n + ":", accuracy_score(df['score'], df['cs'+n]))


Accuracy of cs5: 0.8021259198691741


In [ ]:
cosine_scores.shape

torch.Size([3669, 3669])

In [ ]:
c = cosine_scores

In [ ]:
type(c[0,])

torch.Tensor

In [ ]:
values,indices = c[0,].topk(5)

print(values)
print(indices)

tensor([1.0000, 0.3083, 0.2714, 0.2660, 0.2630])
tensor([1467, 1382, 1930,  825, 1774])


In [ ]:
values,indices = c[1467,].topk(5)
print(values)
print(indices)

In [ ]:
df['s2'].iloc[1467]

'口側の腸管は拡張し 暗赤色を呈していたが 壊死には陥っていなかった'

In [ ]:
df['s1'].iloc[0]

'口側の腸管は拡張し 暗赤色を呈していたが 壊死には陥っていなかった'

In [ ]:
print(indices)

NameError: ignored